In [1]:
import numpy as np
from pygam import s
import pandas as pd
import os

In [18]:
dir_path = r'/Users/christina.bukas/Documents/AI_projects/code/reimplementations/example_data/simple_gam/'
x_path = os.path.join(dir_path, 'X.csv')
b_path = os.path.join(dir_path, 'B.csv')
x = pd.read_csv (x_path, sep=';',header=None)
b = pd.read_csv (b_path, sep=';',header=None)
p_only = pd.read_csv (os.path.join(dir_path, 'P_only.csv'), sep=';',header=None)
p_full = pd.read_csv (os.path.join(dir_path, 'full_P.csv'), sep=';',header=None)
x = x.to_numpy()
#b = b.to_numpy()
#p_only = p_only.to_numpy()
#p_full = p_full.to_numpy()

In [19]:
xx = pd.DataFrame({"a": x[:,0], "b": x[:,1]})

In [39]:
fe =['a', 'b']
xx[fe]

,a,b
0,0.159362,1.820832
1,-0.801953,-1.098624
2,0.898769,1.322290
3,-0.591773,0.217637
4,0.483823,1.811813
...,...,...
995,1.739414,-0.405674
996,0.731487,-1.305809
997,0.904296,1.463110
998,-1.894384,-0.295892


In [34]:
from patsy import dmatrix
formula = '~1+s(a, df=9)+s(b, df=9)'
matrix = dmatrix(formula, xx, return_type='dataframe')

In [35]:
matrix.min()


Intercept                               1.000000
cr(a, df=9, constraints="center")[0]   -0.362168
cr(a, df=9, constraints="center")[1]   -0.360804
cr(a, df=9, constraints="center")[2]   -0.340138
cr(a, df=9, constraints="center")[3]   -0.319788
cr(a, df=9, constraints="center")[4]   -0.364295
cr(a, df=9, constraints="center")[5]   -0.330494
cr(a, df=9, constraints="center")[6]   -0.309615
cr(a, df=9, constraints="center")[7]   -0.407328
cr(a, df=9, constraints="center")[8]   -0.127260
cr(b, df=9, constraints="center")[0]   -0.383915
cr(b, df=9, constraints="center")[1]   -0.375783
cr(b, df=9, constraints="center")[2]   -0.333911
cr(b, df=9, constraints="center")[3]   -0.351432
cr(b, df=9, constraints="center")[4]   -0.338370
cr(b, df=9, constraints="center")[5]   -0.334569
cr(b, df=9, constraints="center")[6]   -0.324953
cr(b, df=9, constraints="center")[7]   -0.382976
cr(b, df=9, constraints="center")[8]   -0.138831
dtype: float64

In [36]:

pd.DataFrame(matrix).to_csv("B_patsy3.csv", sep=';',header=None)

In [21]:
print(p_only.shape, p_full.shape)
print(x.shape)
print(b.shape)

(19, 19) (19, 19)
(1000, 2)
(1000, 19)


In [31]:
from pygam import LinearGAM, l
gam = LinearGAM(s(0) + l(1))

In [36]:
gam.get_params(deep=True)

{'scale': None,
 'max_iter': 100,
 'tol': 0.0001,
 'distribution': NormalDist(),
 'link': 'identity',
 'callbacks': ['deviance', 'diffs'],
 'verbose': False,
 'terms': s(0) + l(1),
 'fit_intercept': True,
 '_constraint_lam': 1000000000.0,
 '_constraint_l2': 0.001,
 '_constraint_l2_max': 0.1,
 '_term_location': 'terms',
 '_name': None,
 '_line_width': 70,
 '_line_offset': 3,
 '_exclude': ['distribution', 'link'],
 '_include': []}

In [52]:
from statsmodels.gam.api import GLMGam, BSplines

In [82]:
s = BSplines(x, df=[12, 10], degree=[3, 3])

In [83]:
s

In [76]:
num_splines = 9

spline1 = s(0, n_splines=num_splines) #, lam=0.2
spline2 = s(1, n_splines=num_splines)

spline1.compile(x)
spline2.compile(x)
m1 = spline1.build_columns(x)
m2 = spline2.build_columns(x)
m1 = m1.toarray()
m2 = m2.toarray()


In [80]:
spline1.info

{'basis': 'ps',
 'n_splines': 9,
 'spline_order': 3,
 'by': None,
 'feature': 0,
 'lam': [0.6],
 'dtype': 'numerical',
 'penalties': ['auto'],
 'constraints': [None],
 'verbose': False,
 'term_type': 'spline_term'}

In [23]:
inter = np.ones((x.shape[0],1))

matrix = np.concatenate([inter, m1, m2], axis=1) # size 1000x19
# x size of 1000x2

ValueError: all the input arrays must have same number of dimensions

In [48]:
p1 = spline1.build_penalties()
p1 = p1.toarray()

p2 = spline2.build_penalties()
p2 = p2.toarray()

p_shape = 1+2*num_splines
p = np.zeros((p_shape, p_shape))
p[1:1+num_splines, 1:1+num_splines] = p1
p[-num_splines:, -num_splines:] = p2

In [53]:
p

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.6, -1.2,  0.6,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -1.2,  3. , -2.4,  0.6,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.6, -2.4,  3.6, -2.4,  0.6,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.6, -2.4,  3.6, -2.4,  0.6,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.6, -2.4,  3.6, -2.4,  0.6,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0.6, -2.4,  3.6, -2.4,  0.6,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.6, -2.4,  3.6, -2.4,  0.6,  0. ,
         

In [61]:
import pandas as pd 
pd.DataFrame(matrix).to_csv("B_gen.csv", sep=';',header=None)
pd.DataFrame(p).to_csv("P_gen.csv", sep=';',header=None)

In [4]:
import pandas as pd 

p_path1 = r'P_gen.csv'
b_path1 = r'B_gen.csv'
p_path2 = r'../example_data/simple_gam/full_P.csv'
b_path2 = r'../example_data/simple_gam/B.csv'

p1 = pd.read_csv (p_path1,sep=';',header=None).values
p1 = p1[:,1:]
p2 = pd.read_csv(p_path2,sep=';',header=None).values
b1 = pd.read_csv(b_path1,sep=';',header=None).values
b2 = pd.read_csv(b_path2,sep=';',header=None).values
b1= b1[:,1:]

In [5]:
p1.shape

(19, 19)

In [6]:
p2.shape

(19, 19)

In [7]:
b1.shape

(1000, 19)

In [8]:
b2.shape

(1000, 19)